In [ ]:
!pip install pyrubberband
!pip install torch-time-stretch

  Preparing metadata (setup.py) ... done
  Created wheel for pyrubberband: filename=pyrubberband-0.3.0-py3-none-any.whl size=4264 sha256=b74c890a60153a89ef0dfc010de9935782b89d138967a3ea2f153af6252d9d73
  Stored in directory: /root/.cache/pip/wheels/b8/2d/f0/bb68fbfe67a42c858a79412321d28589218cbfe114c48ce664
Successfully built pyrubberband
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.5 MB/s eta 0:00:00
     ━

In [ ]:
import torch
import torchaudio
import warnings

from dataclasses import dataclass
from transformers import Wav2Vec2FeatureExtractor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Config,
    Wav2Vec2ForPreTrainingOutput,
    Wav2Vec2GumbelVectorQuantizer,
    Wav2Vec2ForPreTraining
)
from typing import Optional, Tuple, Union

# Extend class, modify forward() to return 'codebook_pairs'
class Wav2Vec2GumbelVectorQuantizerWithCodebookIndices(Wav2Vec2GumbelVectorQuantizer):

    def forward(self, hidden_states, mask_time_indices=None):
        batch_size, sequence_length, hidden_size = hidden_states.shape

        # project to codevector dim
        hidden_states = self.weight_proj(hidden_states)
        hidden_states = hidden_states.view(batch_size * sequence_length * self.num_groups, -1)

        # take argmax in non-differentiable way
        # comptute hard codevector distribution (one hot)
        codevector_idx = hidden_states.argmax(dim=-1)
        codevector_probs = hidden_states.new_zeros(*hidden_states.shape).scatter_(
            -1, codevector_idx.view(-1, 1), 1.0
        )
        codevector_probs = codevector_probs.view(batch_size * sequence_length, self.num_groups, -1)
        codebook_pairs = torch.argmax(codevector_probs, dim=-1)

        perplexity = self._compute_perplexity(codevector_probs, mask_time_indices)

        codevector_probs = codevector_probs.view(batch_size * sequence_length, -1)
        # use probs to retrieve codevectors
        codevectors_per_group = codevector_probs.unsqueeze(-1) * self.codevectors
        codevectors = codevectors_per_group.view(batch_size * sequence_length, self.num_groups, self.num_vars, -1)
        codevectors = codevectors.sum(-2).view(batch_size, sequence_length, -1)

        return codevectors, perplexity, codebook_pairs

# Extend output class to have place for 'codebook_pairs'
@dataclass
class Wav2Vec2ForPreTrainingOutputWithCodebookIndices(Wav2Vec2ForPreTrainingOutput):
    codebook_pairs: Optional[Tuple[torch.FloatTensor]] = None

# Extend class, modify forward() to return 'codebook_pairs'
class Wav2Vec2ForPreTrainingWithCodebookIndices(Wav2Vec2ForPreTraining):
    def __init__(self, config: Wav2Vec2Config):
        super().__init__(config)

        self.quantizer = Wav2Vec2GumbelVectorQuantizerWithCodebookIndices(config)

    def forward(
        self,
        input_values: Optional[torch.Tensor],
        attention_mask: Optional[torch.Tensor] = None,
        mask_time_indices: Optional[torch.BoolTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Wav2Vec2ForPreTrainingOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if mask_time_indices is not None:
            mask_time_indices = mask_time_indices.to(torch.bool)

        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            mask_time_indices=mask_time_indices,
            return_dict=return_dict,
        )

        # 1. project all transformed features (including masked) to final vq dim
        transformer_features = self.project_hid(outputs[0])

        # 2. quantize all (unmasked) extracted features and project to final vq dim
        extract_features = self.dropout_features(outputs[1])

        # Modified quantizer provides 'codebook_pairs' as third item in returned tuple
        quantized_features, codevector_perplexity, codebook_pairs = self.quantizer(
            extract_features, mask_time_indices=mask_time_indices
        )
        quantized_features = self.project_q(quantized_features)

        loss = contrastive_loss = diversity_loss = None

        if not return_dict:
            if loss is not None:
                return (loss, transformer_features, quantized_features, codevector_perplexity) + outputs[2:]
            return (transformer_features, quantized_features, codevector_perplexity) + outputs[2:]

        return Wav2Vec2ForPreTrainingOutputWithCodebookIndices(
            loss=loss,
            projected_states=transformer_features,
            projected_quantized_states=quantized_features,
            codevector_perplexity=codevector_perplexity,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            contrastive_loss=contrastive_loss,
            diversity_loss=diversity_loss,
            codebook_pairs=codebook_pairs
        )

In [ ]:
import torchaudio
librispeech_test = torchaudio.datasets.LIBRISPEECH(".", url="test-clean", download=True)

100%|██████████| 331M/331M [00:14<00:00, 24.3MB/s]


In [ ]:
import os
from scipy import signal
import pyrubberband as pyrb
import torch
import torchaudio
import torchaudio.transforms as T
from torch_time_stretch import time_stretch
import numpy as np
# import shutil
import tempfile
import subprocess
import soundfile as sf

def read_audio(filepath, fs=16000,  mono=True, normalize=False, preemphasis=False):
    """
    Reads audio file stored at <filepath>
    Parameters:
        filepath (str): audio file path
        fs (int, optional): samping rate
        mono (boolean, optional): return single channel
        normalize(boolean, optional): peak normalization of signal
        preemphasis (boolean, optional): apply pre-emphasis filter
    Returns:
        waveform (tensor): audio signal, dim(N,)
    """
    assert isinstance(filepath, str), "filepath must be specified as string"
    assert os.path.exists(filepath), f"{filepath} does not exist."

    try:
        waveform, sr = torchaudio.load(filepath)
        # mono channel
        if waveform.shape[0] == 2 and mono is True: waveform = waveform[0]
        else: waveform = waveform.reshape(-1)
        # preemphasis
        if preemphasis:
            waveform = pre_emphasis(waveform)
        # resample
        if sr != fs:
            resampler = T.Resample(sr, fs, dtype=waveform.dtype)
            waveform = resampler(waveform)
        # normalize
        if normalize:
            waveform = rms_normalize(waveform)
        return waveform
    except Exception as e:
        return None


def peak_normalize(waveform):
    """
    Peak normalizes the <waveform>
    Parameter:
        waveform (tensor): waveform, dims: (N,)
    """
    return waveform/torch.max(torch.abs(waveform))


def rms_normalize(waveform, r=-10):
    """
    RMS-normalization of  <waveform>
    Parameter:
        waveform (tensor): waveform, dims: (N,)
        rms (float): rms in dB
    """
    current_rms = torch.pow(torch.mean(torch.pow(waveform,2)) ,0.5)
    scaling_factor = (10**(r/10))/current_rms
    return waveform*scaling_factor


def pre_emphasis(waveform, coeff=0.97):
    filtered_sig = torch.empty_like(waveform)
    filtered_sig[1:] = waveform[1:] - coeff*waveform[:-1]
    filtered_sig[0] = waveform[0]
    return filtered_sig


def add_time_stretch(audio, fs, stretch_rate):
    """
    Adds time stretch to <clean> audio by <stretch_rate> factor.
    Parameters:
        audio (tensor): waveform, dims: (N,)
        fs (float): audio sample rate
        stretch_rate (float): playback rate
    Returns:
        audio_stretch (tensor): time stretched waveform dims: (N*<stretch_rate>,)

    """
    audio_stretch = time_stretch(audio.unsqueeze(0).unsqueeze(0), 1/stretch_rate, fs)
    # assert len(audio)/stretch_rate == len(audio_stretch), f"stretched audio length mismatch. Expected {len(audio)*stretch_rate}, got {len(audio_stretch)}"
    return audio_stretch.squeeze_()


def add_pitch_shift_rb(y, sr, shift, tmpdir=None):
    """
    Adds pitch shift to <y> audio sampled at <sr> by <shift> semitones. It calls rubberband package directly; does not use pyrubberband package.
    """
    if isinstance(y, np.ndarray) is False:
        y = y.numpy()

    if tmpdir is not None:
        tempfile.tempdir = tmpdir

    # Get the input and output tempfile
    fd, infile = tempfile.mkstemp(suffix='.wav')
    os.close(fd)
    fd, outfile = tempfile.mkstemp(suffix='.wav')
    os.close(fd)

    sf.write(infile, y, sr)
    command = ["rubberband", "-q", "--pitch", str(shift), infile, outfile]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    y_out, _ = sf.read(outfile, always_2d=True, dtype=y.dtype)
    if y.ndim == 1:
        y_out = np.squeeze(y_out)

    os.unlink(infile)
    os.unlink(outfile)
    return torch.from_numpy(y_out).type(torch.float32)


def add_pitch_shift(audio, fs, semitone_shift, use_rb=True):
    """
    Adds pitch shift to <clean> audio by <semitone_shit> semitones.
    Parameters:
        audio (tensor): clean waveform, dims: (N,)
        fs (float): audio sample rate
        semitone_shift (float): semitones (can be between -12 and 12, but keep it low for good sound quality)
    Returns:
        audio_shift: pitch shifted added signal (tensor), dims: (N,)

    """
    if use_rb:
        audio_shift = pyrb.pitch_shift(audio.numpy(), fs, semitone_shift)
        return torch.from_numpy(audio_shift).type(torch.float32)
    else:
        audio_shift = torchaudio.functional.pitch_shift(audio, fs, semitone_shift)
        return audio_shift

def add_noise(audio, noise, snr):
    """
    Adds background <noise> to <clean> signal at desired <SNR> level
    Parameters:
        audio (tensor): clean waveform, dims: (N,)
        noise (tensor): noise waveform, dims: (M,)
        snr (int): SNR level in dB
    Returns:
        noisy_audio: noisy signal (tensor), dims: (N,)
    """
    # make equal lengths for clean and noise signals
    if len(audio) >= len(noise):
        reps = torch.ceil(torch.tensor(len(audio)/len(noise))).int()
        noise = torch.tile(noise, (reps,))[:len(audio)]
    else:
        start_idx = torch.randint(len(noise) - len(audio), (1,))
        noise = noise[start_idx:start_idx+len(audio)]

    assert len(noise) == len(audio), f"noise signal {len(noise)} and clean signal {len(audio)} length mismatch"

    # add noise at desired snr
    audio_rms = torch.mean(torch.pow(audio, 2))
    noise_rms = torch.mean(torch.pow(noise, 2))
    factor = torch.pow((audio_rms/noise_rms)/torch.pow(torch.tensor(10), (snr/10)), 0.5)
    noise = factor*noise
    noisy_audio = audio + noise
    assert 10*torch.log10(audio_rms/torch.mean(torch.pow(noise, 2))) - snr < 1e-4, f"snr mismatch {10*torch.log10(audio_rms/torch.mean(torch.pow(noise, 2))), snr, len(audio), len(noise), audio_rms, torch.mean(torch.pow(noise, 2)), noise_rms, factor, audio, torch.count_nonzero(audio)}"
    return noisy_audio


def add_reverb(clean, rir):
    """
    Filters <clean> signal with <rir> to get reverberation effect
    Parameters:
        clean (tensor): clean waveform, dims: (N,)
        rir (tensor): room impulse response, dims: (M,)
    Returns:
        reverb added signal (tensor), dims: (N,)
    """
    clean = clean.numpy()
    rir = rir.numpy()
    rir = rir/np.linalg.norm(rir)
    # filering
    p_max = np.argmax(np.abs(rir))
    filtered_clean = signal.convolve(clean, rir, mode="full")

    # time offset
    e = np.empty_like(filtered_clean, dtype=np.float32)
    e[-p_max:] = 0.0
    e[:-p_max] = filtered_clean[p_max:]
    # filtered_clean = e.copy()
    # e=None
    filtered_clean = e[:len(clean)]
    assert(len(filtered_clean)==len(clean))
    filtered_clean = torch.from_numpy(filtered_clean)
    return filtered_clean


def add_noise_reverb(audio, noise, snr, rir):
    """
    Adds background <noise> at desired <snr> level and reveberation using <rir> to <clean> signal
    Parameters:
        audio (tensor): clean waveform, dims: (N,)
        noise (tensor): noise waveform, dims: (M,)
        snr (int): SNR level in dB
        rir (tensor): room impulse response, dims: (M,)
    Returns:
        noise and reverb added signal (tensor), dims: (N,)
    """
    audio_reverb = add_reverb(audio, rir)
    noise_reverb = add_reverb(noise, rir)
    noise_reverb_clean = add_noise(audio_reverb, noise_reverb, snr)
    return noise_reverb_clean

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForPreTrainingWithCodebookIndices.from_pretrained("facebook/wav2vec2-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def get_codebook_indices(wav_file):
    # Catch HF gradient_checkpointing deprecation warning
    warnings.filterwarnings(action='ignore', category=UserWarning, module=r'.*configuration_utils')

    if wav_file is None:
        raise ValueError("Error reading audio file.")

    input_values = feature_extractor(wav_file, sampling_rate=16_000, return_tensors="pt").input_values  # Batch size 1

    with torch.no_grad():
        outputs = model(input_values.to(device), output_hidden_states=True, return_dict=True)

    return outputs.codebook_pairs.cpu().numpy()

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForPreTrainingWithCodebookIndices were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

def compare_codebook_pairs(codebook_pairs_clean, codebook_pairs_noisy):
    """
    Compare codebook pairs between clean and noisy audio.
    Parameters:
        codebook_pairs_clean (numpy.ndarray): Codebook pairs for clean audio
        codebook_pairs_noisy (numpy.ndarray): Codebook pairs for noisy audio
    Returns:
        percentage_changed (float): Percentage of codebook pairs changed
        total_tokens (int): Total number of codebook pairs
        num_differences (int): Number of codebook pairs changed
        num_noisy_tokens (int): Number of codebook pairs in noisy audio
    """
    assert isinstance(codebook_pairs_clean, np.ndarray) and isinstance(codebook_pairs_noisy, np.ndarray), "Input must be numpy arrays"
    assert codebook_pairs_clean.shape == codebook_pairs_noisy.shape, "Input arrays must have the same shape"

    total_tokens = codebook_pairs_clean.shape[0]
    num_differences = np.sum(np.any(codebook_pairs_clean != codebook_pairs_noisy, axis=1))
    num_noisy_tokens = total_tokens  # Assuming both arrays have the same number of tokens
    percentage_changed = (num_differences / total_tokens) * 100 if total_tokens > 0 else 0

    return percentage_changed, total_tokens, num_differences, num_noisy_tokens

In [ ]:
# Choose an example from the dataset
example_idx = 0
example_entry = librispeech_test[example_idx]

# Print the entry to understand its structure
print("LibriSpeech Example Entry:", example_entry)

LibriSpeech Example Entry: (tensor([[0.0003, 0.0003, 0.0004,  ..., 0.0021, 0.0021, 0.0016]]), 16000, 'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE', 1089, 134686, 0)


In [ ]:
import glob
# Read paths of all flac files in the directory
flac_files = glob.glob("/content/LibriSpeech/test-clean/**/*.flac", recursive=True)

# Select the first 1000 files
files_subset = flac_files[:10]

print(files_subset)

['/content/LibriSpeech/test-clean/5639/40744/5639-40744-0017.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0034.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0015.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0003.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0036.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0019.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0014.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0011.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0008.flac', '/content/LibriSpeech/test-clean/5639/40744/5639-40744-0016.flac']


In [ ]:
import os
import torch
import torchaudio
import torch.nn.functional as F

def process_single_snr_with_specifications(files_subset, rir_data, offset_ms):
    print(f"\n--- SNR {snr} ---")

    total_percentage_changed = 0  # Initialize total percentage changed for this SNR

    snr_token_counts = 0
    snr_changed_token_counts = 0
    snr_noisy_token_counts = 0

    # Loop over selected audio files
    for file_path in files_subset:
        # Read clean audio
        clean_audio = read_audio(file_path)

        # Add noise to clean audio
        noisy_audio = add_reverb(clean_audio, rir_data)

        # Add time offset to the noisy audio
        if offset_ms != 0:
            sample_rate = 16000  # Assuming 16kHz sample rate, you can adjust according to your case
            offset_samples = int(sample_rate * offset_ms / 1000)
            noisy_audio = F.pad(noisy_audio, (offset_samples, 0))[:-offset_samples]

        # Get codebook pairs for clean and noisy audio
        codebook_pairs_clean = get_codebook_indices(clean_audio)
        codebook_pairs_noisy = get_codebook_indices(noisy_audio)

        # Compare codebook pairs
        percentage_changed, total_tokens, num_differences, num_noisy_tokens = compare_codebook_pairs(
            codebook_pairs_clean, codebook_pairs_noisy)

        # Update total token count for this SNR
        snr_token_counts += total_tokens

        # Update total changed token count for this SNR
        snr_changed_token_counts += num_differences

        # Update total noisy token count for this SNR
        snr_noisy_token_counts += num_noisy_tokens

        # Print total number of clean tokens, number of tokens changed, percentage of tokens changed, and number of noisy tokens
        print(
            f"File: {file_path.split('/')[-1]}, Total clean tokens: {total_tokens}, Tokens changed: {num_differences}, Percentage changed: {percentage_changed:.2f}%, Noisy tokens: {num_noisy_tokens}")

    # Calculate the average percentage of tokens changed for this SNR
    average_percentage_changed_snr = (snr_changed_token_counts / snr_token_counts) * 100 if snr_token_counts > 0 else 0

    # Print the average percentage of tokens changed for this SNR
    print(
        f"Average percentage of tokens changed for SNR {snr}: {average_percentage_changed_snr:.2f}%")

    return snr, snr_token_counts, snr_changed_token_counts, snr_noisy_token_counts, average_percentage_changed_snr

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

snr = 0.1
# Randomly select a noise file
rir_file_path = '/content/0.1.wav'

# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.1 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 278, Percentage changed: 94.88%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 609, Percentage changed: 93.40%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 498, Percentage changed: 90.55%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1263, Percentage changed: 93.21%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 534, Percentage changed: 92.71%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 553, Percentage changed: 91.86%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 350, Percentage changed: 90.91%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 125, Percentage changed: 93.98%, Noisy tokens: 133
File: 5639-40744-000

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

# Randomly select a noise file
rir_file_path = '/content/0.2.wav'

snr = 0.2
# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.2 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 288, Percentage changed: 98.29%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 623, Percentage changed: 95.55%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 523, Percentage changed: 95.09%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1300, Percentage changed: 95.94%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 549, Percentage changed: 95.31%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 580, Percentage changed: 96.35%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 375, Percentage changed: 97.40%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 128, Percentage changed: 96.24%, Noisy tokens: 133
File: 5639-40744-000

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

# Randomly select a noise file
rir_file_path = '/content/0.4.wav'

snr = 0.4
# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.4 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 283, Percentage changed: 96.59%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 629, Percentage changed: 96.47%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 525, Percentage changed: 95.45%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1294, Percentage changed: 95.50%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 555, Percentage changed: 96.35%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 576, Percentage changed: 95.68%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 370, Percentage changed: 96.10%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 131, Percentage changed: 98.50%, Noisy tokens: 133
File: 5639-40744-000

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

# Randomly select a noise file
rir_file_path = '/content/0.5.wav'

snr = 0.5

# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.5 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 290, Percentage changed: 98.98%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 649, Percentage changed: 99.54%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 538, Percentage changed: 97.82%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1336, Percentage changed: 98.60%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 566, Percentage changed: 98.26%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 593, Percentage changed: 98.50%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 380, Percentage changed: 98.70%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 130, Percentage changed: 97.74%, Noisy tokens: 133
File: 5639-40744-000

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

# Randomly select a noise file
rir_file_path = '/content/0.7.wav'

snr = 0.7

# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.7 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 287, Percentage changed: 97.95%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 645, Percentage changed: 98.93%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 532, Percentage changed: 96.73%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1329, Percentage changed: 98.08%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 569, Percentage changed: 98.78%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 585, Percentage changed: 97.18%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 380, Percentage changed: 98.70%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 130, Percentage changed: 97.74%, Noisy tokens: 133
File: 5639-40744-000

In [ ]:
# Select the first 1000 files
files_subset = flac_files[:1000]

# Randomly select a noise file
rir_file_path = '/content/0.8.wav'

snr = 0.8

# Read the noise file
rir_data = read_audio(rir_file_path)

# Specify the offset value in milliseconds
offset_ms = 10  # Example: 10 milliseconds

# Call the function with specifications
snr_result = process_single_snr_with_specifications(files_subset, rir_data, offset_ms)

# Print the result
print(snr_result)


--- SNR 0.8 ---
File: 5639-40744-0017.flac, Total clean tokens: 293, Tokens changed: 292, Percentage changed: 99.66%, Noisy tokens: 293
File: 5639-40744-0034.flac, Total clean tokens: 652, Tokens changed: 651, Percentage changed: 99.85%, Noisy tokens: 652
File: 5639-40744-0015.flac, Total clean tokens: 550, Tokens changed: 541, Percentage changed: 98.36%, Noisy tokens: 550
File: 5639-40744-0003.flac, Total clean tokens: 1355, Tokens changed: 1349, Percentage changed: 99.56%, Noisy tokens: 1355
File: 5639-40744-0036.flac, Total clean tokens: 576, Tokens changed: 571, Percentage changed: 99.13%, Noisy tokens: 576
File: 5639-40744-0019.flac, Total clean tokens: 602, Tokens changed: 596, Percentage changed: 99.00%, Noisy tokens: 602
File: 5639-40744-0014.flac, Total clean tokens: 385, Tokens changed: 383, Percentage changed: 99.48%, Noisy tokens: 385
File: 5639-40744-0011.flac, Total clean tokens: 133, Tokens changed: 131, Percentage changed: 98.50%, Noisy tokens: 133
File: 5639-40744-000